In [10]:
# Load necessary libraries
library(readxl)
library(pracma)  # for trapz()

# Read the Excel file
data <- read_excel("prcsd_trial2.xlsx")

time <- data[[1]]

# Function to calculate doubling time from a vector of values
calc_doubling_time <- function(time, values) {
  # Filter out NA and zero or negative values for log calculation
  valid_idx <- which(!is.na(values) & values > 0)
  if(length(valid_idx) < 2) return(NA)
  
  log_values <- log(values[valid_idx])
  time_vals <- time[valid_idx]
  
  # Linear regression on log-transformed data to find growth rate
  fit <- lm(log_values ~ time_vals)
  rate <- coef(fit)[2]
  
  if(rate <= 0) return(NA)
  
  doubling_time <- log(2) / rate
  return(doubling_time)
}

# Loop through columns (excluding time)
results <- data.frame(Column = character(), AUC = numeric(), DoublingTime = numeric(), stringsAsFactors=FALSE)

for (colname in names(data)[-1]) {
  values <- data[[colname]]
  # Calculate AUC using trapezoidal rule
  auc <- tryCatch({
    trapz(time, values)
  }, error = function(e) NA)
  
  # Calculate doubling time
  dt <- calc_doubling_time(time, values)
  
  results <- rbind(results, data.frame(Column = colname, AUC = auc, DoublingTime = dt))
}

print(results)
write.csv(results, "growth_summary.csv", row.names = FALSE)

                         Column      AUC DoublingTime
time_vals       D1 Rep1 Average 19.41521     3.383633
time_vals1     D1 Rep 2 Average 19.10932     2.979688
time_vals2     D1 Rep 3 Average 19.57633     2.877215
time_vals3     AE2 Rep1 Average 15.41025     3.021794
time_vals4     AE2 Rep2 Average 14.71322     3.163587
time_vals5     AE2 Rep3 Average 15.20122     3.348050
time_vals6     110 Rep1 Average 15.84185     3.894459
time_vals7     110 Rep2 Average 11.67468     2.429639
time_vals8     110 Rep3 Average 14.38459     2.750515
time_vals9     111 Rep1 Average 17.34116     3.455855
time_vals10    111 Rep2 Average 12.96821     2.466949
time_vals11    111 Rep3 Average 15.16690     2.633706
time_vals12    112 Rep1 Average 13.88328     2.511881
time_vals13    112 Rep2 Average 16.43176     2.505138
time_vals14 CBS138 Rep1 Average       NA     4.464452
time_vals15 CBS138 Rep2 Average       NA     2.788375
time_vals16 CBS138 Rep3 Average 15.89303     3.265279
time_vals17 CBS138 Rep4 Aver

In [ ]:
#fix Na Values
nafix_df <- read_excel('NaN_fix.xlsx')
nafix_df <- nafix_df[nafix_df[[1]] != 24.00000, ]
tail(nafix_df)
# Assume first column is time
time <- nafix_df[[1]]

# Prepare results
auc_results <- data.frame(Column = character(), AUC = numeric(), stringsAsFactors = FALSE)

# Loop through numeric columns (excluding time)
for (colname in names(nafix_df)[-1]) {
  values <- nafix_df[[colname]]
  if (is.numeric(values)) {
    # Replace NA with 0 for integration
    values[is.na(values)] <- 0
    auc <- trapz(time, values)
    auc_results <- rbind(auc_results, data.frame(Column = colname, AUC = auc))
  }
}

print(auc_results)
# I manually replpaced the NaN values in the growth_summary.csv file

Time,CBS138 Rep1 Average,CBS138 Rep2 Average,BG2 Rep1 Average
<dbl>,<dbl>,<dbl>,<dbl>
22.00000,0.9955988,1.012875,1.154725
22.33334,0.9945988,1.013125,1.155217
22.66667,0.9931988,1.012625,1.155567
23.00000,0.9919988,1.012500,1.155558
23.33334,1.0117500,1.011750,1.230187
23.66667,1.0112500,1.011250,1.232063


               Column      AUC
1 CBS138 Rep1 Average 17.24843
2 CBS138 Rep2 Average 15.83786
3    BG2 Rep1 Average 20.28257
